# Algoritmo Branch and Bound

## Setup

In [77]:
import pdb
import numpy as np
from copy import deepcopy as copy
from ortools.linear_solver import pywraplp
import random
Z_max = -np.inf
new_solver = pywraplp.Solver.CreateSolver

# variaveis auxiliares
infinity = np.inf

# funcoes auxiliares
def all_variables_are_integer(variables):
    for i in range(len(variables)):
        if variables[i].solution_value() % 1 != 0:
            return False
    return True

def pick_variable_and_return_its_lower_bound(variables):
    for i in range(len(variables)):
        value = variables[i].solution_value()
        if value % 1 != 0:
             return (i, np.floor(value))
        
def create_left_subproblem(node, variable, lower_bound):
    left = copy(node)
    new_constraint = [0 for i in range(left['relaxed_LP']['num_vars'])]
    new_constraint[variable] = 1
    left['relaxed_LP']['constraint_coeffs'].append(
        new_constraint
    )
    left['relaxed_LP']['num_constraints'] += 1
    left['relaxed_LP']['bounds'].append(lower_bound)
    return left

def create_right_subproblem(node, variable, upper_bound):
    right = copy(node)
    new_constraint = [0 for i in range(right['relaxed_LP']['num_vars'])]
    new_constraint[variable] = -1
    right['relaxed_LP']['constraint_coeffs'].append(
        new_constraint
    )
    right['relaxed_LP']['num_constraints'] += 1
    right['relaxed_LP']['bounds'].append(-upper_bound)
    return right

def add_leq_restrictions(solver, relaxed_LP, x):
    for i in range(relaxed_LP['num_constraints']):
        if relaxed_LP['bounds'][i] >= 0:
            constraint = solver.RowConstraint(
                0, relaxed_LP['bounds'][i]
            )
            for j in range(relaxed_LP['num_vars']):
                constraint.SetCoefficient(
                    x[j], relaxed_LP['constraint_coeffs'][i][j]
                )
    return solver

def add_geq_restrictions(solver, relaxed_LP, x):
    for i in range(relaxed_LP['num_constraints']):
        if relaxed_LP['bounds'][i] < 0:
            constraint = solver.RowConstraint(
                -relaxed_LP['bounds'][i], infinity
            )
            for j in range(relaxed_LP['num_vars']):
                if relaxed_LP['constraint_coeffs'][i][j] < 0:
                    constraint.SetCoefficient(
                        x[j], -relaxed_LP['constraint_coeffs'][i][j]
                    )
                else:
                    constraint.SetCoefficient(
                        x[j], relaxed_LP['constraint_coeffs'][i][j]
                    )
    return solver

def create_solver(relaxed_LP):
    # definindo solver
    solver = new_solver('GLOP')
    # adicionando variaveis
    x = {}
    for i in range(relaxed_LP['num_vars']):
        x[i] = solver.NumVar(0, infinity, 'x_{}'.format(i))
    # adicionando restricoes menor ou igual
    solver = add_leq_restrictions(solver, relaxed_LP, x)
    # adicionando restricoes maior ou igual
    solver = add_geq_restrictions(solver, relaxed_LP, x)
    # definindo o objetivo
    objective = solver.Objective()
    for i in range(relaxed_LP['num_vars']):
        objective.SetCoefficient(
            x[i], relaxed_LP['obj_coeffs'][i]
        )
    objective.SetMaximization()
    # retornando o solver e suas variaveis
    return solver

### Branch and Bound

$$
\begin{align*}
\text{max }  & 3x_0 + x_1 \\
\text{s.a. } & 4x_0 + 2x_1 \leq 11 \\
             & x_i \in Z_{+} \forall i = 0, 1
\end{align*}
$$

In [78]:
# criando arvore de branch and bround
optimal_solution = [None, None]
optimal_value = -infinity
root = {
    'relaxed_LP': {
        'num_vars': 2,
        'num_constraints': 1,
        'constraint_coeffs': [
            [4, 2]
        ],
        'bounds': [11],
        'obj_coeffs': [3, 1]
    },
    'left': None,
    'right': None
}

# criando fila de problemas branch and bound a serem resolvidos
Q = [root]

iterations = 0
# iterando na fila de problemas até que tenhamos resolvido todos
while len(Q) > 0:
    # 1. seleciono (e removo) o problema do topo da fila para ser resolvido
    node = Q.pop()
    # crio um solver a partir deste problema
    solver = create_solver(node['relaxed_LP'])
    # resolvo este problema
    status = solver.Solve()
    
    # se o problema nao for factivel, descarto ele da minha pilha
    if status != pywraplp.Solver.OPTIMAL:
        continue
        
    # avalio se as solucoes do solver respeitam a restrição
    # de integridade inteira
    variables = solver.variables()
    Z = solver.Objective().Value()
    if all_variables_are_integer(variables):
        # se o valor da funcao objetivo
        # do solver é maior que o valor otimo atual para limitar
        # (bound) o problema
        if Z > optimal_value:
            # atualizo optimal_value
            optimal_value = Z
            # atualizo optimal_solution
            for i in range(len(variables)):
                optimal_solution[i] = variables[i].solution_value()
    # se não respeitam a restrição de integridade
    # seleciono a primeira variavel que viola a restricao
    # para particionar (branch) o problema
    else:
        variable, lower_bound = pick_variable_and_return_its_lower_bound(variables)
        upper_bound = lower_bound + 1
        # crio subproblema a esquerda
        # adicionando a restricao variable <= lower_bound
        left = create_left_subproblem(node, variable, lower_bound)
        # crio subproblema a direita
        # adicionando a restricao variable >= upper_bound
        right = create_right_subproblem(node, variable, upper_bound)
        # adiciono left e right ao node
        node['left'] = left
        node['right'] = right
        # para otimizar o branch and bound, adiciono na pilha de problemas
        # o problema com menor valor, para em sequencia adicionar o de
        # maior valor, que vai ser processado primeiro
        feasible_nodes = []
        feasible_nodes_values = []
        # resolvo o problema da esquerda
        left_solver = create_solver(left['relaxed_LP'])
        left_status = left_solver.Solve()
        # se o problema da esquerda for factivel
        # e seu valor de funcao objetivo
        # for maior que o valor otimo encontrado
        # até o momento (bound)
        # faço sua inserção na lista de problemas 
        # a serem adicionados na pilha Q
        if left_status == pywraplp.Solver.OPTIMAL and left_solver.Objective().Value() > optimal_value:
        #if left_status == pywraplp.Solver.OPTIMAL:
            feasible_nodes.append(left)
            feasible_nodes_values.append(left_solver.Objective().Value())
        # resolvo o problema da direita
        right_solver = create_solver(right['relaxed_LP'])
        right_status = right_solver.Solve()
        # se o problema da direita for factivel
        # e seu valor de funcao objetivo
        # for maior que o valor otimo encontrado
        # até o momento (bound)
        # faço sua inserção na lista de problemas 
        # a serem adicionados na pilha Q
        if right_status == pywraplp.Solver.OPTIMAL and right_solver.Objective().Value() > optimal_value:
        #if right_status == pywraplp.Solver.OPTIMAL:
            feasible_nodes.append(right)
            feasible_nodes_values.append(right_solver.Objective().Value())
        # defino uma funcao de ordenacao para ordenar feasible solutions
        # pelo valor da funcao objetivo, de forma que o que tiver
        # maior valor de objetivo seja resolvido primeiro
        sort_by_objective_value = lambda x: x[1]
        nodes_to_enqueue = [
            node for node, node_value in sorted(
                zip(
                    feasible_nodes, feasible_nodes_values
                ),
                key = sort_by_objective_value
            )
        ]
        # enfileiro os subproblemas para serem resolvidos
        # resolvendo primeiro os que tiverem maior
        # valor objetivo
        Q += nodes_to_enqueue
        iterations += 1
print(optimal_value)
print(optimal_solution)
print(iterations)

7.0
[2.0, 1.0]
6


### Comparando com o ortools do google

$$
\begin{align*}
\text{max }  & 3x_0 + x_1 \\
\text{s.a. } & 4x_0 + 2x_1 \leq 11 \\
             & x_i \in Z_{+} \forall i = 0, 1
\end{align*}
$$

In [79]:
from ortools.linear_solver import pywraplp
solver = pywraplp.Solver.CreateSolver('SCIP')
x_0 = solver.IntVar(0, infinity, 'x_0')
x_1 = solver.IntVar(0, infinity, 'x_1')
solver.Add(4*x_0 + 2*x_1 <= 11)
solver.Maximize(3*x_0 + x_1)
status = solver.Solve()
if status == pywraplp.Solver.OPTIMAL:
    print('Solution:')
    print('Objective value =', solver.Objective().Value())
    print('x_0 =', x_0.solution_value())
    print('x_1 =', x_1.solution_value())
else:
    print('The problem does not have an optimal solution.')

Solution:
Objective value = 7.0
x_0 = 2.0
x_1 = 1.0


### Branch and Bound

$$
\begin{align*}
\text{max }  & 7 x_1 + 8 x_2 + 2 x_3 + 9 x_4 + 6 x_5 \\
\text{s.a. } & 5 x_1 + 7 x_2 + 9 x_3 + 2 x_4 + 1 x_5 \leq 250 \\
             & 18x_1 + 4 x_2 - 9 x_3 + 10 x_4 + 12 x_5 \leq 285 \\
             &   x_1 + 7 x_2 + 3 x_3 + 8 x_4 + 5 x_5 \leq 211 \\
             &   x_1 + 13 x_2 + 16 x_3 + 3 x_4 - 7 x_5 \leq 315 \\
             &   x_i \in Z_{+} \forall i = 1, \cdots, 5
\end{align*}
$$

In [76]:
# criando arvore de branch and bround
optimal_solution = [None, None, None, None, None]
optimal_value = -infinity
data={}
data['constraint_coeffs'] = [
        [5, 7, 9, 2, 1],
        [18, 4, -9, 10, 12],
        [4, 7, 3, 8, 5],
        [5, 13, 16, 3, -7],
    ]
data['bounds'] = [250, 285, 211, 315]
data['obj_coeffs'] = [7, 8, 2, 9, 6]
data['num_vars'] = 5
data['num_constraints'] = 4
root = {
    'relaxed_LP': data,
    'left': None,
    'right': None
}

# criando fila de problemas branch and bound a serem resolvidos
Q = [root]

iterations = 0
# iterando na fila de problemas até que tenhamos resolvido todos
while len(Q) > 0:
    # 1. seleciono (e removo) o problema do topo da fila para ser resolvido
    node = Q.pop()
    # crio um solver a partir deste problema
    solver = create_solver(node['relaxed_LP'])
    # resolvo este problema
    status = solver.Solve()
    
    # se o problema nao for factivel, descarto ele da minha pilha
    if status != pywraplp.Solver.OPTIMAL:
        continue
        
    # avalio se as solucoes do solver respeitam a restrição
    # de integridade inteira
    variables = solver.variables()
    Z = solver.Objective().Value()
    if all_variables_are_integer(variables):
        # se o valor da funcao objetivo
        # do solver é maior que o valor otimo atual para limitar
        # (bound) o problema
        if Z > optimal_value:
            # atualizo optimal_value
            optimal_value = Z
            # atualizo optimal_solution
            for i in range(len(variables)):
                optimal_solution[i] = variables[i].solution_value()
    # se não respeitam a restrição de integridade
    # seleciono a primeira variavel que viola a restricao
    # para particionar (branch) o problema
    else:
        variable, lower_bound = pick_variable_and_return_its_lower_bound(variables)
        upper_bound = lower_bound + 1
        # crio subproblema a esquerda
        # adicionando a restricao variable <= lower_bound
        left = create_left_subproblem(node, variable, lower_bound)
        # crio subproblema a direita
        # adicionando a restricao variable >= upper_bound
        right = create_right_subproblem(node, variable, upper_bound)
        # adiciono left e right ao node
        node['left'] = left
        node['right'] = right
        # para otimizar o branch and bound, adiciono na pilha de problemas
        # o problema com menor valor, para em sequencia adicionar o de
        # maior valor, que vai ser processado primeiro
        feasible_nodes = []
        feasible_nodes_values = []
        # resolvo o problema da esquerda
        left_solver = create_solver(left['relaxed_LP'])
        left_status = left_solver.Solve()
        # se o problema da esquerda for factivel
        # e seu valor de funcao objetivo
        # for maior que o valor otimo encontrado
        # até o momento (bound)
        # faço sua inserção na lista de problemas 
        # a serem adicionados na pilha Q
        if left_status == pywraplp.Solver.OPTIMAL and left_solver.Objective().Value() > optimal_value:
        #if left_status == pywraplp.Solver.OPTIMAL:
            feasible_nodes.append(left)
            feasible_nodes_values.append(left_solver.Objective().Value())
        # resolvo o problema da direita
        right_solver = create_solver(right['relaxed_LP'])
        right_status = right_solver.Solve()
        # se o problema da direita for factivel
        # e seu valor de funcao objetivo
        # for maior que o valor otimo encontrado
        # até o momento (bound)
        # faço sua inserção na lista de problemas 
        # a serem adicionados na pilha Q
        if right_status == pywraplp.Solver.OPTIMAL and right_solver.Objective().Value() > optimal_value:
        #if right_status == pywraplp.Solver.OPTIMAL:
            feasible_nodes.append(right)
            feasible_nodes_values.append(right_solver.Objective().Value())
        # defino uma funcao de ordenacao para ordenar feasible solutions
        # pelo valor da funcao objetivo, de forma que o que tiver
        # maior valor de objetivo seja resolvido primeiro
        sort_by_objective_value = lambda x: x[1]
        nodes_to_enqueue = [
            node for node, node_value in sorted(
                zip(
                    feasible_nodes, feasible_nodes_values
                ),
                key = sort_by_objective_value
            )
        ]
        # enfileiro os subproblemas para serem resolvidos
        # resolvendo primeiro os que tiverem maior
        # valor objetivo
        Q += nodes_to_enqueue
        iterations += 1
print(optimal_value)
print(optimal_solution)
print(iterations)

260.0
[9.0, 20.0, 2.0, 1.0, 4.0]
97


### Comparando com OR Tools do Google

$$
\begin{align*}
\text{max }  & 7 x_1 + 8 x_2 + 2 x_3 + 9 x_4 + 6 x_5 \\
\text{s.a. } & 5 x_1 + 7 x_2 + 9 x_3 + 2 x_4 + 1 x_5 \leq 250 \\
             & 18x_1 + 4 x_2 - 9 x_3 + 10 x_4 + 12 x_5 \leq 285 \\
             &   x_1 + 7 x_2 + 3 x_3 + 8 x_4 + 5 x_5 \leq 211 \\
             &   x_1 + 13 x_2 + 16 x_3 + 3 x_4 - 7 x_5 \leq 315 \\
             &   x_i \in Z_{+} \forall i = 1, \cdots, 5
\end{align*}
$$

In [75]:
from ortools.linear_solver import pywraplp


def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['constraint_coeffs'] = [
        [5, 7, 9, 2, 1],
        [18, 4, -9, 10, 12],
        [4, 7, 3, 8, 5],
        [5, 13, 16, 3, -7],
    ]
    data['bounds'] = [250, 285, 211, 315]
    data['obj_coeffs'] = [7, 8, 2, 9, 6]
    data['num_vars'] = 5
    data['num_constraints'] = 4
    return data

def main():
    data = create_data_model()
    # Create the mip solver with the SCIP backend.
    solver = pywraplp.Solver.CreateSolver('SAT')
    if not solver:
        return

    infinity = solver.infinity()
    x = {}
    for j in range(data['num_vars']):
        x[j] = solver.IntVar(0, infinity, 'x[%i]' % j)
    print('Number of variables =', solver.NumVariables())

    for i in range(data['num_constraints']):
        constraint = solver.RowConstraint(0, data['bounds'][i], '')
        for j in range(data['num_vars']):
            constraint.SetCoefficient(x[j], data['constraint_coeffs'][i][j])
    print('Number of constraints =', solver.NumConstraints())
    # In Python, you can also set the constraints as follows.
    # for i in range(data['num_constraints']):
    #  constraint_expr = \
    # [data['constraint_coeffs'][i][j] * x[j] for j in range(data['num_vars'])]
    #  solver.Add(sum(constraint_expr) <= data['bounds'][i])

    objective = solver.Objective()
    for j in range(data['num_vars']):
        objective.SetCoefficient(x[j], data['obj_coeffs'][j])
    objective.SetMaximization()
    # In Python, you can also set the objective as follows.
    # obj_expr = [data['obj_coeffs'][j] * x[j] for j in range(data['num_vars'])]
    # solver.Maximize(solver.Sum(obj_expr))

    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        print('Objective value =', solver.Objective().Value())
        for j in range(data['num_vars']):
            print(x[j].name(), ' = ', x[j].solution_value())
        print()
        print('Problem solved in %f milliseconds' % solver.wall_time())
        print('Problem solved in %d iterations' % solver.iterations())
        print('Problem solved in %d branch-and-bound nodes' % solver.nodes())
    else:
        print('The problem does not have an optimal solution.')
main()

Number of variables = 5
Number of constraints = 4
Objective value = 260.0
x[0]  =  9.0
x[1]  =  20.0
x[2]  =  2.0
x[3]  =  1.0
x[4]  =  4.0

Problem solved in 21.000000 milliseconds
Problem solved in 0 iterations
Problem solved in 0 branch-and-bound nodes
